In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from catboost import Pool, CatBoostRegressor, CatBoost, CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scipy.stats import randint as randint
from scipy.stats import uniform
from sklearn.model_selection import cross_val_score
import re # Регулярные выражения.
import string
import datetime
import pymystem3

try:
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import GridSearchCV
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold


ohe = OneHotEncoder(sparse = False)
mystem=pymystem3.Mystem()
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)
def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y

    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())

Для вас представлена задача, сделанная на открытых данных.
Вам нужно предсказать цену товара по другим имеющимся характеристикам.
Метрика качества RMSLE.<br>
ЦЕНА в КОПЕЙКАХ!!
Но предсказания нужны в РУБЛЯХ
потому - сразу поделите в train прайс на 100

In [2]:
df_train = pd.read_pickle('./data/train_sample.pckl')

In [96]:
df_test = pd.read_pickle('./data/test_hack.pckl')

In [97]:
!telegram-send 'success'

In [98]:
df_test.head()

,can_buy,can_promote,category,contacts_visible,date_created,delivery_available,description,fields,id,images,location,mortgage_available,name,payment_available,subcategory,subway
4,False,False,6,True,1517061944,True,"Стремянка трехсекционная,3×4","[{'field': {'name': 'Ремонт и строительство', ...",285ea2e9935ccdeb8378c6a5,"[{'id': '5a6c864a074b3e7d207f3522', 'num': 1, ...","{'latitude': 54.749969, 'longitude': 55.993774}",False,Стремянка,True,603,NaN
9,False,False,2,True,1508310389,False,Плита ЭВИ 5120. Работают 3 конфорки и духовка....,"[{'field': {'name': 'Бытовая техника', 'id': 2...",adfb73820bbb831257df6e95,"[{'id': '59e6fc49f202633e173ad413', 'num': 1, ...","{'latitude': 59.737736, 'longitude': 30.078828}",False,Плита электрическая,True,203,NaN
15,False,False,1,True,1517089590,False,Все лоты привезены с площадок и аукционов Япон...,"[{'field': {'name': 'Транспорт и автотовары', ...",783025601c36202f633fc6a5,"[{'id': '5a6cf2b65eaa9e4973249b62', 'num': 1, ...","{'latitude': 55.989156, 'longitude': 37.145419}",False,Диски R17 Rays MazdaSpeed Touring 5х114.3 7J,False,116,NaN
19,False,False,10,True,1509378271,False,,"[{'field': {'name': 'Хобби и развлечения', 'id...",2f0cd2d2e15dc90afd847f95,"[{'id': '59f748bca09cd5081f40d786', 'num': 1, ...","{'latitude': 55.805178, 'longitude': 37.771612}",False,Batman Archam Knight,True,1009,NaN
20,False,False,11,True,1515487857,False,Отдаю ледобур времён с.с.с.р диаметр 130 мм бе...,"[{'field': {'name': 'Спорт и отдых', 'id': 11,...",5c23a37902855a20172845a5,"[{'id': '5a548186c6ab9e73a2056162', 'num': 1, ...","{'latitude': 59.843029, 'longitude': 30.246231}",False,Ледобур для зимней рыбалки,False,1104,NaN


In [ ]:
df_train[:100]

In [ ]:
df_train.info()

In [99]:
def replace_logical(s):
    if s:
        return 1
    else:
        return 0
def transformYear(val):
    #year = datetime.datetime.fromtimestamp(int(df_train.date_created[0])).year
    year = datetime.datetime.fromtimestamp(int(val)).year
    return year

def transformMonth(val):
    month = datetime.datetime.fromtimestamp(val).month
    return month

def get_latitude(val):
    res = val['latitude']
    return res

def get_longitude(val):
    res = val['longitude']
    return res

def preprocess_data(df_input):
    df_output = df_input
    df_output = pd.get_dummies(df_output)
    return df_output

def data_initial_preproc (df_input):
    
    df_output = df_input
    #df_output['price'] = df_output['price']/100
    df_output['can_buy'] = df_output['can_buy'].map(lambda s: replace_logical(s))
    df_output['can_promote'] = df_output['can_promote'].map(lambda s: replace_logical(s))
    df_output['contacts_visible'] = df_output['contacts_visible'].map(lambda s: replace_logical(s))
    df_output['mortgage_available'] = df_output['mortgage_available'].map(lambda s: replace_logical(s))
    df_output['delivery_available'] = df_output['delivery_available'].map(lambda s: replace_logical(s))
    df_output['payment_available'] = df_output['payment_available'].map(lambda s: replace_logical(s))
    df_output['year']=df_output['date_created'].map(transformYear)
    df_output['month']=df_output['date_created'].map(transformMonth)
    df_output = df_output.drop(['date_created', 'description', 'fields', 'id', 'images', 'subway'], axis = 1)
    df_output = df_output.drop(['can_buy', 'can_promote', 'category'], axis = 1)
    #df_output = df_output[df_output['price']>0]
    df_output = df_output.drop(['name'], axis = 1)
    df_output['latitude'] = df_output['location'].map(get_latitude)
    df_output['longitude'] = df_output['location'].map(get_longitude)
    df_output = df_output.drop(['location'],axis = 1)
    #df_output['price'] = np.log10(df_output['price'] + 1)
    df_output['subcategory'] = df_output['subcategory'].map(lambda s: 'S' + str(s))
    df_output = df_output.pipe(preprocess_data)
    
    return df_output

In [101]:
df_test_preproc = df_test.pipe(data_initial_preproc)

In [102]:
df_test_preproc

,contacts_visible,delivery_available,mortgage_available,payment_available,year,month,latitude,longitude,subcategory_S10001,subcategory_S1001,...,subcategory_S907,subcategory_S908,subcategory_S909,subcategory_S910,subcategory_S911,subcategory_S912,subcategory_S914,subcategory_S919,subcategory_S920,subcategory_S921
4,1,1,0,1,2018,1,54.749969,55.993774,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,1,2017,10,59.737736,30.078828,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,0,0,2018,1,55.989156,37.145419,0,0,...,0,0,0,0,0,0,0,0,0,0
19,1,0,0,1,2017,10,55.805178,37.771612,0,0,...,0,0,0,0,0,0,0,0,0,0
20,1,0,0,0,2018,1,59.843029,30.246231,0,0,...,0,0,0,0,0,0,0,0,0,0
21,1,1,0,1,2018,1,55.982010,37.861309,0,0,...,0,0,0,0,0,0,0,0,0,0
24,1,0,0,0,2017,12,56.851392,53.112731,0,0,...,0,0,0,0,0,0,0,0,0,0
28,1,0,0,1,2017,12,57.198030,65.589105,0,0,...,0,0,0,0,0,0,0,0,0,0
29,1,0,0,1,2017,10,55.808347,37.798014,0,0,...,0,1,0,0,0,0,0,0,0,0
31,1,0,0,0,2018,2,56.968020,40.998946,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
import pickle
model = pickle.load(open('model.pckl', 'rb'))

/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [124]:
idx = df_test_preproc.columns != 'Churn?'
feature_names = df_test_preproc.columns[idx]
importances = pd.Series(index=feature_names, data=model.feature_importances_).sort_values()

In [125]:
importances.sort_


contacts_visible      0.000000e+00
mortgage_available    0.000000e+00
subcategory_S814      1.135987e-07
subcategory_S1414     1.626163e-06
subcategory_S1603     1.798432e-06
subcategory_S816      4.599834e-06
subcategory_S1609     4.850192e-06
subcategory_S710      9.425223e-06
subcategory_S415      1.006399e-05
subcategory_S1115     1.181985e-05
subcategory_S213      1.185063e-05
subcategory_S1511     1.448672e-05
subcategory_S1209     1.989342e-05
subcategory_S1114     1.993801e-05
subcategory_S1413     2.111408e-05
subcategory_S1211     2.458448e-05
subcategory_S317      2.524087e-05
subcategory_S1615     2.767333e-05
subcategory_S610      2.773540e-05
subcategory_S707      2.792716e-05
subcategory_S708      3.228211e-05
subcategory_S704      3.575579e-05
subcategory_S1510     4.512164e-05
subcategory_S1210     4.582938e-05
subcategory_S215      4.644050e-05
subcategory_S809      4.731046e-05
subcategory_S414      4.775514e-05
subcategory_S1612     4.888707e-05
subcategory_S1711   

In [110]:
prediction = model.predict(df_test_preproc)

In [111]:
prediction

array([3.3192824 , 3.65316057, 3.23486309, ..., 2.9314511 , 3.48814208,
       3.10256722])

In [112]:
10**prediction - 1

array([2084.84675315, 4498.46183928, 1716.36689265, ...,  852.98668642,
       3076.10335665, 1265.38925182])

In [114]:
result = 10**prediction - 1

In [116]:
submission = pd.DataFrame()
submission.loc[:,'id'] = df_test.reset_index()['id']
submission.loc[:,'price'] = result
submission.to_csv('submission.csv', sep=',', index=False, encoding='utf-8')